In [1]:
from tqdm import tqdm
import numpy as np
import pandas as pd


In [3]:
df = pd.read_csv('data/cf_ba_combined.csv')

In [ ]:
from transformers import BertTokenizerFast, BertModel
checkpoint = 'unikei/bert-base-proteins'
tokenizer = BertTokenizerFast.from_pretrained(checkpoint)
model = BertModel.from_pretrained(checkpoint)

## MHC

In [7]:
mhc = df['mhc_seq'].unique()
mhc_name = df['mhc_name'].unique()

In [ ]:
from tqdm import tqdm
import numpy as np
mhc_dict = {}
mhc_arr = []
mhc_arr_2d = []

for idx, row in df.iterrows():
    mhc_dict[row['mhc_name']] = row['mhc_seq']
for name, m in tqdm(mhc_dict.items()):
    mhc_arr.append(m)
tokens = tokenizer(mhc_arr, return_tensors='pt', padding=True)
output = model(**tokens)[0].detach().cpu()
mhc_arr_2d = np.array(output)
np.save('data/mhc_2d_lm.npy', mhc_arr_2d)

## peptide

In [10]:
pep = df['peptide'].unique()

In [12]:
pt_arr_2d = []
pt = []
ptl = 16
for p in pep:
    pt.append(p[:ptl])

In [ ]:
b = 500
for i in tqdm(range(0, len(pep), b)):
    cur = pt[i:i+b]
    tokens = tokenizer(cur, return_tensors='pt', padding='max_length', truncation=True, max_length=ptl)
    output = model(**tokens)[0].detach().cpu()
    pt_arr_2d.extend(np.array(output))

In [14]:
np.save('data/pt_2d_lm.npy', np.array(pt_arr_2d))